In [2]:
import pandas as pd
import calendar
import numpy as np
from sklearn.decomposition import PCA
import statsmodels.api as sm
from sklearn.cross_decomposition import PLSRegression
from datetime import datetime

In [3]:
# loading dataframes 

r_daily = pd.read_csv('managed_portfolios_anom_d_55.csv')

# drop all momentum factors or factors that are constructed based on momentum

factor_drop_list = ['r_mom', 'r_indmom', 'r_valmom', 'r_valmomprof', 'r_mom12', 'r_momrev', 'r_indmomrev', 'r_exchsw', 'rme', 're_ew']

r_daily.drop(columns=factor_drop_list, inplace=True)

# set date to datetime format and set the date to the index 

r_daily['date'] = pd.to_datetime(r_daily['date'])
r_daily.set_index('date', inplace=True)

# following the procedure in the paper, if there are observations missing we set them to 0. 

r_daily.fillna(0, inplace=True) 

# create a list of factors for later analysis purposes 

factors = [col for col in r_daily.columns if col.startswith('r_')]

# create a monthly return dataframe for later analysis purposes 

r_monthly = r_daily.resample('M').sum()
r_monthly.index = r_monthly.index.strftime('%Y-%m')


In [4]:
# initialize pca model 

pca = PCA(n_components=len(factors))

# select our start date 

start_date = pd.to_datetime("1963-07-01")

# create an empty dataframe to store the average return for each PC from t until t-11

pc_avg_df = pd.DataFrame()

# create an empty list for the pc return dataframes. These will be concated in a later stage to one large dataframe

pc_return_dfs = []

# create our loop set up, this is actually an expanding PCA analysis. In each iteration a new month is added to the dataset and the return is computed. 

for year in range(1973, 2020):
    # as our out of sample procedure starts July 1973, we start in the 7th month in 1973
    for mo in range(6,13) if year == 1973 else range(1, 13):
        # first we have to find the last month of the day. For this we use the calender function with inputs from the loop variables
        last_day = calendar.monthrange(year, mo)[1]

        # we select our new end_date variable for which the PCA analysis is done, also with inputs from our loop and the last_day variable
        end_date = pd.to_datetime(f'{year}-{mo}-{last_day}')

        t_dt = pd.to_datetime(f'{year}-{mo}')
        t = t_dt.strftime('%Y-%m')
        
        # we select the datarange from our dataset (July 1963 = start_date until our defined end_date) and we fit the model
        pca_data = r_daily.loc[start_date:end_date]
        pca.fit(pca_data)

        # we extract the principal components. These principal components are put in a new dataframe for later analysis. 
        
        principal_components = pca.components_
        components_df = pd.DataFrame(data=principal_components.T, index=factors, columns=[f"PC{i+1}" for i in range(len(factors))])

        # calculating return for month t+1. If mo = 12, then year will increment with 1. 
        
        t_plus_1_year = year + 1 if mo == 12 else year
        t_plus_1_month = (mo % 12) + 1
 
        # creating a datetime variable for the month t+1 and storing this in our pc_return_data variable
        
        t_plus_1_dt =pd.to_datetime(f'{t_plus_1_year}-{t_plus_1_month}')
        t_plus_1 = t_plus_1_dt.strftime('%Y-%m')
        
        pc_return_data = {'date': t_plus_1}

        
        # in this loop we calculate the monthly factor returns (f) using the principal components and returns
        
        for f in range(len(factors)):
            # select our factor and extract its principal component from principal_df and its return from r_daily for all observations in month mo 
            pc = components_df.iloc[:, f]
            r_month = r_monthly.loc[t]
            # multiply the principal components with the returns and sum them up to get PC factor return for month mo 
            pc_return = (pc*r_month).sum()
            
            # place this in our dictionary for later transposing to dataframe
            
            pc_return_data[components_df.columns[f]] = pc_return

            r_pc_month_n_list = []
            
            for n in range(1, 12):
                # calculate the datetime for t - n
                t_minus_n_dt = t_dt - pd.DateOffset(months=n)

                # transpose it to our YYYY-MM format
                t_minus_n = t_minus_n_dt.strftime('%Y-%m')

                # select the return corresponding to our month t-n
                r_month_n = r_monthly.shift(n).loc[t_minus_n]
                
                pc_return_n = (pc*r_month_n).sum()
                
                r_pc_month_n_list.append(pc_return_n)

            r_pc_month_mean = (np.mean(r_pc_month_n_list))
            pc_avg_df.loc[t, f'PC{f+1}'] = r_pc_month_mean
                

        pc_return_df = pd.DataFrame.from_dict(pc_return_data, orient='index').T
        pc_return_df.set_index('date', inplace=True)
        pc_return_dfs.append(pc_return_df)

r_pc = pd.concat(pc_return_dfs)
print(r_pc)


              PC1       PC2       PC3       PC4       PC5       PC6       PC7  \
date                                                                            
1973-07 -0.107861  0.064028 -0.001788  0.058085 -0.068429 -0.028958  0.098368   
1973-08  0.528398  0.069094  0.025934  0.029249  0.298858   0.11805  0.150817   
1973-09 -0.057159 -0.025929  0.013906  0.017154  0.024258  0.022785  0.004781   
1973-10  0.026133 -0.091035  0.143031   0.10927  0.051393 -0.002975  0.062329   
1973-11 -0.249047  -0.05311  0.091882  0.073315 -0.039798  0.089521  0.081162   
...           ...       ...       ...       ...       ...       ...       ...   
2019-09  0.064238 -0.266023 -0.044962  0.143513  0.054072 -0.064805 -0.022804   
2019-10 -0.416507  0.057316  0.239891  -0.06507 -0.072235  0.014179  0.070646   
2019-11  0.036042 -0.110662  0.011783 -0.062658  0.021307   0.08512  0.050528   
2019-12  0.203826  0.043237 -0.041169  0.013195 -0.030844  0.045996 -0.009277   
2020-01  -0.07257  0.050186 

### Leveraging and demeaning our returns

In [5]:
start_date_dt = pd.to_datetime("1963-07-01")
start_date = start_date_dt.strftime('%Y-%m')
lev_dfs = []

for year in range(1973, 2020):
    # as our out of sample procedure starts July 1973, we start in the 8th month in 1973 as we need to demean our PC factor up to month t
    for mo in range(7,13) if year == 1973 else range(1, 13):
        
        # first we set our t variable to the current year and month from our loop
        
        t_dt = pd.to_datetime(f'{year}-{mo}')
        t_minus_one_dt = t_dt - pd.DateOffset(months=1)
        t = t_dt.strftime('%Y-%m')
        t_minus_one = t_minus_one_dt.strftime('%Y-%m')
        
        
        

        # calculate the variance of the individual factor returns up until month t 
        r_indiv_f_t = r_monthly.loc[start_date:t_minus_one]
        var_indiv_f_t = r_indiv_f_t.var(axis=0)
        avg_var_indiv_f_t = var_indiv_f_t.mean()
        #print(r_indiv_f_t.mean())

        # calculate the mean and variance of the PC factors up until month t 

        r_pc_t = r_pc.loc[:t]
        #print(r_pc_t)
        demeaned_r_pc_t = r_pc.loc[t].to_frame().T - r_pc_t.mean()

        # calculate the leverage factor and multiply this with the demeaned 
        
        leverage_t = np.sqrt(avg_var_indiv_f_t / r_pc_t.std(axis=0))
        lev_r_pc_t = demeaned_r_pc_t * leverage_t
        lev_df = lev_r_pc_t.loc[t].to_frame().T
        lev_dfs.append(lev_df)

lev_r_pc = pd.concat(lev_dfs)
lev_r_pc.fillna(0, inplace=True)
lev_r_pc_clean = lev_r_pc.drop(lev_r_pc.index[:1])
print(lev_r_pc_clean)

              PC1       PC2       PC3       PC4       PC5       PC6       PC7  \
1973-08  0.018475  0.001649  0.003856 -0.003933  0.014037  0.008881  0.005305   
1973-09 -0.011645 -0.010358  0.000403 -0.004734 -0.005389 -0.002063 -0.011406   
1973-10 -0.005119 -0.013320  0.014773  0.010718 -0.002466 -0.004635 -0.002626   
1973-11 -0.019847 -0.006679  0.005894  0.003244 -0.009523  0.007803  0.000283   
1973-12 -0.025570 -0.008397 -0.015948 -0.002359 -0.025697  0.000194 -0.022067   
...           ...       ...       ...       ...       ...       ...       ...   
2019-08  0.007440 -0.003935 -0.001149  0.006904 -0.008900  0.003877  0.007561   
2019-09  0.008861 -0.038551 -0.008395  0.022178  0.009162 -0.012681 -0.004893   
2019-10 -0.041758  0.006949  0.041047 -0.013787 -0.014275  0.002868  0.014021   
2019-11  0.005952 -0.016681  0.001406 -0.013342  0.003101  0.016810  0.009923   
2019-12  0.023596  0.004986 -0.007782 -0.000252 -0.006578  0.009087 -0.002193   

              PC8       PC9

In [17]:

positive_returns_PC = pc_avg_df > 0
negative_returns_PC = pc_avg_df < 0 


long_portfolio_PC = positive_returns_PC.astype(int)  
short_portfolio_PC = negative_returns_PC.astype(int) 

mom_1_10 = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10']
mom_11_20 = ['PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18', 'PC19', 'PC20']
mom_21_30 = ['PC21', 'PC22', 'PC23', 'PC24', 'PC25', 'PC26', 'PC27', 'PC28', 'PC29', 'PC30']
mom_31_40 = ['PC31', 'PC32', 'PC33', 'PC34', 'PC35', 'PC36', 'PC37', 'PC38', 'PC39', 'PC40']
mom_41_47 = ['PC41', 'PC42', 'PC43', 'PC44', 'PC45', 'PC46', 'PC47']

mom_list = [mom_1_10, mom_11_20, mom_21_30, mom_31_40, mom_41_47]

r_set_dict = {}

# Calculate and accumulate the results for each momentum set
for i, mom in enumerate(mom_list):
    # Calculate returns for long positions minus returns for short positions
    r_PC_set_mom = (long_portfolio_PC[mom] * lev_r_pc_clean[mom]) - (short_portfolio_PC[mom] * lev_r_pc_clean[mom])
    # No mean calculation here; just accumulate the returns
    r_set_dict[f'mom_set_{i + 1}'] = r_PC_set_mom

# Concatenate the results into a single DataFrame
mom_strategy = pd.concat(r_set_dict, axis=1)

# Adjust the index to date format and then to 'YYYY-MM' string format
mom_strategy.index = pd.to_datetime(mom_strategy.index)
mom_strategy.index = mom_strategy.index.strftime('%Y-%m')



print(mom_strategy)
# If you still want to view the mean return of each set for overview purposes
print("Mean returns of each momentum set:")
print(mom_strategy.mean(axis=0))

        mom_set_1                                                              \
              PC1       PC2       PC3       PC4       PC5       PC6       PC7   
1973-06       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
1973-07       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
1973-08 -0.018475  0.001649 -0.003856 -0.003933 -0.014037  0.008881 -0.005305   
1973-09 -0.011645 -0.010358 -0.000403 -0.004734 -0.005389 -0.002063 -0.011406   
1973-10  0.005119 -0.013320 -0.014773  0.010718  0.002466 -0.004635  0.002626   
...           ...       ...       ...       ...       ...       ...       ...   
2019-08  0.007440  0.003935  0.001149  0.006904  0.008900  0.003877 -0.007561   
2019-09  0.008861  0.038551  0.008395  0.022178 -0.009162  0.012681 -0.004893   
2019-10 -0.041758 -0.006949 -0.041047 -0.013787  0.014275  0.002868 -0.014021   
2019-11  0.005952  0.016681  0.001406 -0.013342 -0.003101 -0.016810  0.009923   
2019-12  0.023596 -0.004986 

### Replicate table 3 panel A 

In [18]:
mom_strategy_1 = mom_strategy.loc[:'1996-09']
mom_strategy_2 = mom_strategy.loc['1996-09':]

print(f'the mean of every subset of PCs is:\n')
print(mom_strategy.mean(axis=0))

means = mom_strategy.mean(axis=0).tolist()
std = mom_strategy.std(axis=0).tolist()
N = mom_strategy.shape[0]

print(f'the t-statistic of every subset of PCS is:\n')
for m, s in zip(means, std):
    t_statistic = m / (s / (N**0.5))
    print(t_statistic)

print(f'the mean of every subset of PCs is (first half):\n')
print(mom_strategy_1.mean(axis=0))

means = mom_strategy_1.mean(axis=0).tolist()
std = mom_strategy_1.std(axis=0).tolist()
N = mom_strategy_1.shape[0]

print(f'the t-statistic of every subset of PCS is (first half):\n')
for m, s in zip(means, std):
    t_statistic = m / (s / (N**0.5))
    print(t_statistic)

print(f'the mean of every subset of PCs is (second half):\n')
print(mom_strategy_2.mean(axis=0))

means = mom_strategy_2.mean(axis=0).tolist()
std = mom_strategy_2.std(axis=0).tolist()
N = mom_strategy_2.shape[0]

print(f'the t-statistic of every subset of PCS is (second half):\n')
for m, s in zip(means, std):
    t_statistic = m / (s / (N**0.5))
    print(t_statistic)


the mean of every subset of PCs is:

mom_set_1  PC1     1.667545e-04
           PC2     5.316020e-04
           PC3    -4.214299e-04
           PC4    -4.307215e-04
           PC5     2.024112e-03
           PC6     4.778123e-04
           PC7     1.674533e-03
           PC8     1.151123e-03
           PC9     8.977791e-04
           PC10    4.042838e-04
mom_set_2  PC11    1.822154e-03
           PC12    1.065264e-03
           PC13    3.645756e-05
           PC14   -3.555906e-06
           PC15    8.480714e-04
           PC16    1.278196e-04
           PC17    2.188431e-04
           PC18    5.326006e-04
           PC19    6.325922e-04
           PC20    1.859322e-04
mom_set_3  PC21    3.442104e-04
           PC22    1.051316e-03
           PC23    8.615438e-05
           PC24    2.088472e-04
           PC25    3.013740e-04
           PC26    4.962097e-04
           PC27    2.557393e-04
           PC28   -1.836344e-04
           PC29    5.524860e-04
           PC30   -1.514939e-04
mom

### Replicatie tabel 3 panel B en C 

In [9]:
ff = pd.read_stata("fffactors.dta")

ff.set_index('yyyymm', inplace=True)


ff.index = pd.to_datetime(ff.index, format='%Y%m')


ff.index = ff.index.strftime('%Y-%m')

ff5 = ff[['mktrf', 'smb', 'hml', 'rmw', 'cma']].loc['1973-08':'2019-12']

mom_strategy_ff5 = pd.concat([mom_strategy, ff5], axis=1)

mom_strategy_ff5['P1'] = 0
mom_strategy_ff5['P2'] = 0
mom_strategy_ff5.loc[mom_strategy_ff5.index <= '1996-09', 'P1'] = 1
mom_strategy_ff5.loc[mom_strategy_ff5.index >= '1996-09', 'P2'] = 1




FileNotFoundError: [Errno 2] No such file or directory: 'fffactors.dta'

### Panel B

In [8]:
independent_vars = ['mom_set_1', 'P1', 'P2', 'mktrf', 'smb', 'hml', 'rmw', 'cma']
X = mom_strategy_ff5[independent_vars]

for i in range(2, 6):
    Y = mom_strategy_ff5[f'mom_set_{i}']
    modelB = sm.OLS(y, X).fit()
    print(modelB.summary())



NameError: name 'mom_strategy_ff5' is not defined

### Panel C

In [140]:
y = mom_strategy_ff5['mom_set_1']

for i in range(2, 6):
    independent_vars = [f'mom_set_{i}', 'P1', 'P2', 'mktrf', 'smb', 'hml', 'rmw', 'cma']
    X = sm.add_constant(mom_strategy_ff5[independent_vars])
    modelC = sm.OLS(y, X).fit()
    print(modelC.summary())


independent_vars = ['P1', 'P2', 'mom_set_2', 'mom_set_3', 'mom_set_4', 'mom_set_5', 'mktrf', 'smb', 'hml', 'rmw', 'cma']
   

                            OLS Regression Results                            
Dep. Variable:              mom_set_1   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     1.945
Date:                Thu, 22 Feb 2024   Prob (F-statistic):             0.0514
Time:                        00:33:08   Log-Likelihood:                 1496.7
No. Observations:                 557   AIC:                            -2975.
Df Residuals:                     548   BIC:                            -2936.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0093      0.017      0.555      0.5